In [113]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

### Merge dataframes

In [232]:
df_list = []
for year in range(1996, 2022):
    df = pd.read_csv('Data/' + str(year) + '-' + str(year + 1)[-2:] + '_stats.csv')
    df_list.append(df)
df_merged = pd.concat(df_list)
df_merged[['FG_PCT', 'FG3_PCT', 'PTS']]
X_train, X_test, y_train, y_test = train_test_split(df_merged[['FG_PCT', 'FG3_PCT', 'PTS']], df_merged['W_PCT_x'], test_size = 0.33, random_state = None)
X_train

,FG_PCT,FG3_PCT,PTS
23,0.461,0.407,96.2
16,0.441,0.331,99.6
16,0.478,0.355,106.5
10,0.440,0.342,92.6
14,0.478,0.345,99.9
...,...,...,...
6,0.444,0.344,102.3
29,0.435,0.365,93.2
2,0.475,0.361,112.9
28,0.475,0.349,103.6


### Linear Regression

In [240]:
model = LinearRegression()

In [251]:
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df_merged[['REB']], df_merged['W_PCT_x'], test_size = 0.25, random_state=None)
X_train_list.append(X_train_1), X_test_list.append(X_test_1), y_train_list.append(y_train_1), y_test_list.append(y_test_1)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_merged[['REB']], df_merged['W_PCT_x'], test_size = 0.25, random_state=None)
X_train_list.append(X_train_2), X_test_list.append(X_test_2), y_train_list.append(y_train_2), y_test_list.append(y_test_2)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(df_merged[['REB']], df_merged['W_PCT_x'], test_size = 0.25, random_state=None)
X_train_list.append(X_train_3), X_test_list.append(X_test_3), y_train_list.append(y_train_3), y_test_list.append(y_test_3)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(df_merged[['REB']], df_merged['W_PCT_x'], test_size = 0.25, random_state=None)
X_train_list.append(X_train_4), X_test_list.append(X_test_4), y_train_list.append(y_train_4), y_test_list.append(y_test_4)

(None, None, None, None)

In [252]:

def rmse(targets, predictions):
    return np.sqrt(np.mean(np.square(targets - predictions)))

def Linear_Regression(X_train, X_test, y_train, y_test):
    # Create and train model
    model = LinearRegression().fit(X_train, y_train)

    # Generate predictions
    predictions = model.predict(X_test)

    # Compute loss to evaluate model
    loss = rmse(y_test, predictions)
    return loss

loss = []
var_val = 0
for i in range(0, 4):
    loss.append(Linear_Regression(X_train_list[i], X_test_list[i], y_train_list[i], y_test_list[i]))

avg_val_acc = sum(loss)/4
print("avg validation accuracy: " + str(avg_val_acc))
print("variance validation accuracuy: " + str(np.var(loss)))

avg validation accuracy: 0.1531536928607806
variance validation accuracuy: 2.256326561517804e-05


### Neural Network

In [253]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size1, output_size):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.layer3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu1(out)
        out = self.layer2(out)
        out = self.relu2(out)
        out = self.layer3(out)
        return out
    
input_size = 1
hidden_size1 = 31
output_size = 1
net = Net(input_size, hidden_size1, output_size)

loss_fn = nn.MSELoss()

optimizer = optim.SGD(net.parameters(), lr=0.01)
X = torch.Tensor(X_train_1.values)
Y = torch.Tensor(y_train_1.values)

In [254]:
losses = []
num_epochs = 400
batch_size = 31
for split in range(4):
    X_train = X_train_list[split]
    y_train = y_train_list[split]
    X_test = X_test_list[split]
    y_test = y_test_list[split]
    
    for epoch in range(num_epochs):
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            
            # Clear gradients of optimizer
            optimizer.zero_grad()
            
            # Forward pass
            y_pred = net(torch.Tensor(X_batch.values))
            
            # Compute loss
            loss = loss_fn(y_pred, torch.Tensor(y_batch.values))
            
            # Backward pass
            loss.backward()
            
            # Update parameters
            optimizer.step()
            
        print("Epoch: " + str(epoch))
        print("Loss: " + str(loss**(1/2)))
    y_pred = net(torch.Tensor(X_test.values))
    loss_test = loss_fn(y_pred, torch.Tensor(y_test.values))
    losses.append(loss_test.detach().numpy())
losses_sqrt = np.sqrt(losses)
avg_val_acc = sum(losses_sqrt)/4
print("avg validation accuracy: " + str(avg_val_acc))
print("variance validation accuracuy: " + str(np.var(losses_sqrt)))

C:\Users\broha\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\broha\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0
Loss: tensor(6.3218e+13, grad_fn=<PowBackward0>)
Epoch: 1
Loss: tensor(4.3066e+13, grad_fn=<PowBackward0>)
Epoch: 2
Loss: tensor(2.9338e+13, grad_fn=<PowBackward0>)
Epoch: 3
Loss: tensor(1.9986e+13, grad_fn=<PowBackward0>)
Epoch: 4
Loss: tensor(1.3615e+13, grad_fn=<PowBackward0>)
Epoch: 5
Loss: tensor(9.2751e+12, grad_fn=<PowBackward0>)
Epoch: 6
Loss: tensor(6.3185e+12, grad_fn=<PowBackward0>)
Epoch: 7
Loss: tensor(4.3044e+12, grad_fn=<PowBackward0>)
Epoch: 8
Loss: tensor(2.9323e+12, grad_fn=<PowBackward0>)
Epoch: 9
Loss: tensor(1.9976e+12, grad_fn=<PowBackward0>)
Epoch: 10
Loss: tensor(1.3608e+12, grad_fn=<PowBackward0>)
Epoch: 11
Loss: tensor(9.2702e+11, grad_fn=<PowBackward0>)
Epoch: 12
Loss: tensor(6.3152e+11, grad_fn=<PowBackward0>)
Epoch: 13
Loss: tensor(4.3021e+11, grad_fn=<PowBackward0>)
Epoch: 14
Loss: tensor(2.9307e+11, grad_fn=<PowBackward0>)
Epoch: 15
Loss: tensor(1.9965e+11, grad_fn=<PowBackward0>)
Epoch: 16
Loss: tensor(1.3601e+11, grad_fn=<PowBackward0>)
Epoch: 

Epoch: 150
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 151
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 152
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 153
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 154
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 155
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 156
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 157
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 158
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 159
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 160
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 161
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 162
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 163
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 164
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 165
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 166
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 167
Loss: tensor(0.1862, grad_fn=<PowBack

Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 305
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 306
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 307
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 308
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 309
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 310
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 311
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 312
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 313
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 314
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 315
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 316
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 317
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 318
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 319
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 320
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epoch: 321
Loss: tensor(0.1862, grad_fn=<PowBackward0>)
Epo

C:\Users\broha\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([193])) that is different to the input size (torch.Size([193, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 55
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 56
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 57
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 58
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 59
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 60
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 61
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 62
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 63
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 64
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 65
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 66
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 67
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 68
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 69
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 70
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 71
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 72
Loss: tens

Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 207
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 208
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 209
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 210
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 211
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 212
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 213
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 214
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 215
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 216
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 217
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 218
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 219
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 220
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 221
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 222
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 223
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epo

Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 358
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 359
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 360
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 361
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 362
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 363
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 364
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 365
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 366
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 367
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 368
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 369
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 370
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 371
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 372
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 373
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epoch: 374
Loss: tensor(0.1468, grad_fn=<PowBackward0>)
Epo

Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 109
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 110
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 111
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 112
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 113
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 114
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 115
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 116
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 117
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 118
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 119
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 120
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 121
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 122
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 123
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 124
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 125
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epo

Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 260
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 261
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 262
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 263
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 264
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 265
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 266
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 267
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 268
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 269
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 270
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 271
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 272
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 273
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 274
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 275
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epoch: 276
Loss: tensor(0.1496, grad_fn=<PowBackward0>)
Epo

### Resume

In [ ]:
# Look through first two sections under Research Experience, and look at real world examples to reformat it
# Projects: Look at real world examples for projects, how they format theirs and do the same with yours. 
# Leadership: Include JIRA, look at other leadership for resume see which intimiates you and include on yours